In [1]:
import numpy as np
from foe_foundry_nl.embeddings import embeddings, similarity, distance
from foe_foundry_nl.data.monsters import get_canonical_monsters, name_to_key, SrdMonsterSelector
from foe_foundry.creature_types import CreatureType
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt


sns.set_style("whitegrid")
sns.set_palette("hls", len(CreatureType.all()))

In [2]:
monsters = get_canonical_monsters()
embeddings = embeddings(list(monsters.keys()), skip_if_missing=True)

In [ ]:
tsne = TSNE(n_components=2, random_state=20240711)
X = tsne.fit_transform(embeddings.embeddings)

x1, x2 = X[:, 0], X[:, 1]
ct = [monsters[k].creature_type for k in embeddings.keys]

df = pd.DataFrame([
    pd.Series(x1, name='x1'),
    pd.Series(x2, name='x2'),
    pd.Series(ct, name='creature_type')
]).T

fig, ax = plt.subplots(1,1, figsize=(16,9))
sns.scatterplot(
    data=df,
    x='x1',
    y='x2',
    hue='creature_type',
    ax=ax
)


In [ ]:
pca = PCA(n_components=2)
X = pca.fit_transform(embeddings.embeddings)
x1, x2 = X[:, 0], X[:, 1]

df = pd.DataFrame([
    pd.Series(x1, name='x1'),
    pd.Series(x2, name='x2'),
    pd.Series(ct, name='creature_type')
]).T

fig, ax = plt.subplots(1,1, figsize=(18,12))
sns.scatterplot(
    data=df,
    x = 'x1',
    y= 'x2',
    hue='creature_type'
)

iconic_srd_monsters = [m.key for _, m in monsters.items() if m.is_srd]

srd_xs, srd_ys, srd_cts = [], [], []
for key in iconic_srd_monsters:
    index = embeddings.key_index(key)
    embedding = embeddings[index]
    pca_x = pca.transform(embedding[np.newaxis, :])
    m_x, m_y = pca_x[0, 0], pca_x[0, 1]
    srd_xs.append(m_x)
    srd_ys.append(m_y)
    srd_cts.append(monsters[key].creature_type)

ax.scatter(
    x=srd_xs,
    y=srd_ys,
    color='k',
    marker='x'
)

for i, monster in enumerate(iconic_srd_monsters):
    ax.annotate(monster, (srd_xs[i], srd_ys[i]), size=6)


for srd_monster in iconic_srd_monsters:
    key = name_to_key(srd_monster)
    index = embeddings.key_index(key)
    embedding = embeddings[index]

    mask = np.ones(embeddings.n, dtype=np.bool)
    mask[index] = False
    others = embeddings.embeddings[mask,:]

    similarities = similarity(embedding[np.newaxis, :], others)
    similarities = similarities.flatten()

    distances = distance(embedding[np.newaxis, :], others)
    distances = distances.flatten()

    # indexes = np.argsort(similarities)
    # furthest = indexes[:3]
    # closest = indexes[-3:]
    # closest_keys = embeddings.keys[closest].tolist()
    # furthest_keys = embeddings.keys[furthest].tolist()
    # print(f"{key} is closest to {','.join(closest_keys)} and furthest from {','.join(furthest_keys)}")

    # indexes = np.argsort(distances)
    # furthest = indexes[-3:]
    # closest = indexes[:3]
    # closest_keys = embeddings.keys[closest].tolist()
    # furthest_keys = embeddings.keys[furthest].tolist()
    # print(f"{key} is closest to {','.join(closest_keys)} and furthest from {','.join(furthest_keys)}")


fig, ax = plt.subplots(1,1, figsize=(18,12))
df = pd.DataFrame([
    pd.Series(srd_xs, name='x1'),
    pd.Series(srd_ys, name='x2'),
    pd.Series(srd_cts, name='creature_type')
]).T

sns.scatterplot(
    data=df,
    x='x1',
    y='x2',
    hue='creature_type',
    ax=ax
)

for i, monster in enumerate(iconic_srd_monsters):
    ax.annotate(monster, (srd_xs[i], srd_ys[i]), size=6)




In [2]:
import numpy as np
from foe_foundry_nl.data.monsters import SrdMonsterSelector
rng = np.random.default_rng(20240711)
selector = SrdMonsterSelector(rng)


for _ in range(100):
    anchor, positive, negative = selector.random_positive_negative_triplet()
    print(f"{anchor=}, {positive=}, {negative=}")



anchor='green_hag', positive='sea_hag', negative='ettin'
anchor='giant_goat', positive='quipper', negative='sprite'
anchor='magma_mephit', positive='dust_mephit', negative='adult_blue_dragon'
anchor='reef_shark', positive='plesiosaurus', negative='animated_armor'
anchor='venomous_snake', positive='weasel', negative='rakshasa'
anchor='duergar', positive='mage', negative='giant_constrictor_snake'
anchor='ancient_copper_dragon', positive='young_copper_dragon', negative='lich'
anchor='salamander', positive='earth_elemental', negative='killer_whale'
anchor='brown_bear', positive='polar_bear', negative='blink_dog'
anchor='elephant', positive='goat', negative='aboleth'
anchor='ape', positive='giant_ape', negative='ancient_blue_dragon'
anchor='couatl', positive='sphinx_of_wonder', negative='ancient_green_dragon'
anchor='zombie', positive='pit_fiend', negative='adult_green_dragon'
anchor='wraith', positive='minotaur_skeleton', negative='giant_wasp'
anchor='merrow', positive='giant_eagle', negat

In [ ]:
from foe_foundry_nl.data.monsters import get_canonical_monsters
monsters = get_canonical_monsters()
subtypes = set()
for _, m in monsters.items():
    subtypes.update(m.creature_subtypes)
print(subtypes)
